Koden under er en versjon som kjører synkront.

In [ ]:
import time

def make_meal(delay, what):
    print(what, end=": ")
    time.sleep(delay)
    print('Ferdig!')

def main():
    print(f"Startet {time.strftime('%X')}")
    
    make_meal(3, 'Lage kaffe')
    make_meal(5, 'Koke egg')
    make_meal(3, 'Steike bacon')
    make_meal(5, 'Varme bønner i tomatsaus')
    make_meal(3, 'Riste brød')
    make_meal(1, 'Sette på bordet')

    print('Frokosten er ferdig!')
    print(f"Ferdig ved {time.strftime('%X')}")

main()

Og nå benytter vi oss av asyncio for å asynkront kjøre jobbene i parallell

In [ ]:
import asyncio
import time

async def make_meal(delay, what):
    await asyncio.sleep(delay)
    print(what, ": Ferdig!")

async def main() :

    print(f"Startet {time.strftime('%X')}")
    await asyncio.gather(make_meal(3, 'Lage kaffe'),
        make_meal(5, 'Koke egg'),
        make_meal(3, 'Steike bacon'),
        make_meal(5, 'Varme bønner i tomatsaus'),
        make_meal(3, 'Riste brød'),
        make_meal(1, 'Sette på bordet'))
    print(f"Ferdig ved {time.strftime('%X')}")

await main()


Skal man utveksle informasjon på tvers av jobber er det naturlig å gjøre dette via en kø. 
Eksempel på producer/consumer applikasjon som samhandler via en slik kø: En restaurant med litt spesielle regler. 
1. 3 kelnere tar kun ordre på hver sin type mat.
2. Restauranten stenger når kokken har greid å tømme ordre-boka.

In [ ]:
import asyncio
import random
import time

class Order :
    def __init__(self, meal, how_long) :
        self.__meal = meal
        self.__how_long = how_long
    
    @property
    def meal(self) :
        return self.__meal
    
    @property
    def how_long(self) :
        return self.__how_long
      

async def waiter(meal, how_long, queue) :
    # Place order in the queue
    while True :
        print("Legger matrett av typen", meal, "inn til bestilling")
        await queue.put(Order(meal, how_long))  # Will block if queue is full TEST: put_nowait()
        await asyncio.sleep(random.uniform(0,12))

async def cheaf(queue) :
    while not queue.empty() :
        order = await queue.get()
        print("Lager matrett av type", order.meal, "ved å bruke", order.how_long, "sekunder. Køen er nå", queue.qsize(), "stor")
        await asyncio.sleep(order.how_long)
        print("Ferdig med å lage", order.meal)
        queue.task_done()

    print("**** Vi stenger kjøkkenet! *****")



async def main() :
    queue = asyncio.Queue(10)

    kelner1 = asyncio.create_task(waiter("Kjøttsuppe",1, queue))
    kelner2 = asyncio.create_task(waiter("Biff",2, queue))
    kelner3 = asyncio.create_task(waiter("Grøt",3, queue))
  
    consumer = asyncio.create_task(cheaf(queue))
    producers = [kelner1, kelner2, kelner3]
    # asyncio.run()

    await queue.join()
    await asyncio.gather(consumer)
    for kelner in producers :
        kelner.cancel()

    print('====')

await main()  

Et eksempel på hvordan man kan jobbe med IO i dette biblioteket (hentet fra https://realpython.com/python-concurrency/).

In [ ]:
import asyncio
import time
import aiohttp

async def download_site(session, url):
    async with session.get(url) as response: # Bruk av context-manager 
        print("Read {0} from {1}".format(response.content_length, url))


async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            task = asyncio.create_task(download_site(session, url))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=True)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    # asyncio.run(download_all_sites(sites))
    await download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} sites in {duration} seconds")

In [ ]:
import time
import requests

def download_site(session, url):
    with session.get(url) as response:
        print("Read {0} from {1}".format(len(response.content), url))

def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            download_site(session, url)

if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    # asyncio.run(download_all_sites(sites))
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} sites in {duration} seconds")